In [1]:
import json
from datetime import datetime, timedelta,date
import pandas as pd
from functools import reduce
import numpy as np 
import os.path

class ReportTM:
    
    def generate_DF(self,name_file):
        with open('data/'+name_file, encoding='utf-8') as fh:
            data = json.load(fh)

        price_date_name = 'currentPrice_'+str(name_file[:-5])
        Data_products = {
            "id" : [],
            'colorDescription' : [],
            'portrait_url' : [],
            'squarishURL' : [],
            'Stock' : [],
            'isNew' : [],
            price_date_name : [],
            'fullPrice' : [],
            'subtitle' : [],
            'title' : [],
            'date':[],
            'url':[]
        }

        for product in data:
            Data_products['id'].append(product['id'])
            Data_products['colorDescription'].append(product['colorDescription'])
            Data_products['portrait_url'].append(product['images']['portraitURL'])
            Data_products['squarishURL'].append(product['images']['squarishURL'])
            Data_products['Stock'].append(product['inStock'])
            Data_products['isNew'].append(product['colorways'][0]['isNew'])
            Data_products[price_date_name].append(product['price']['currentPrice'])
            Data_products['fullPrice'].append(product['price']['fullPrice'])
            Data_products['subtitle'].append(product['subtitle'])
            Data_products['title'].append(product['title'])
            Data_products['date'].append(name_file[:-5])
            Data_products['url'].append(product['url'])
        
        return pd.DataFrame(Data_products)

    def generate_time_serie(self,*files):
        array_dataframes =[]
        for file in files:
            with open('data/'+file, encoding='utf-8') as fh:
                data = json.load(fh)

            price_date_name = file[:-5]
            Data_products = {
                "id" : [],
                price_date_name : []

            }

            for product in data:
                Data_products['id'].append(product['id'])
                Data_products[price_date_name].append(product['price']['currentPrice'])
            array_dataframes.append(pd.DataFrame(Data_products))

        # merge a la lista de dataframes

        df_final = reduce(lambda left,right: pd.merge(left,right,on='id', how='outer'), array_dataframes)

        return df_final
    
    def check_changes_2days(self, day):
    
        #obtenemos el dia anterior a la fecha dada
        date = datetime.strptime(day, "%Y-%m-%d")
        modified_date = date - timedelta(days=1)
        prev_day = datetime.strftime(modified_date, "%Y-%m-%d")
        #comprobamos que existan archivos para los dos dias
        day_file = day +'.json'
        prev_day_file = prev_day+'.json'
        if os.path.isfile('data/'+day_file) and os.path.isfile('data/'+prev_day_file) :
            df_time = self.generate_time_serie(prev_day_file,day_file)
            df_time['change'] = df_time[prev_day_file[:-5]] - df_time[day_file[:-5]]
            return df_time.loc[df_time['change']>0]
            #
        else:
            raise ValueError(
                    "Nombres de archivo no validos. "
                        f"Only one file per ZIP: {prev_day_file,day_file}"
                    )
            
    

In [21]:
import glob
#obtenemos todos los dataframes port archivo en el folder
list_dataf = glob.glob("data/*.json")
list_dataf = [z[5:] for z in list_dataf]
list_dataf

['2020-06-27.json',
 '2020-06-28.json',
 '2020-06-29.json',
 '2020-06-30.json',
 '2020-07-01.json',
 '2020-07-02.json',
 '2020-07-03.json',
 '2020-07-04.json',
 '2020-07-05.json']

In [22]:

info= ReportTM()
dataframes = []
#guardamos los dataframes
for data in list_dataf:
    dataframes.append(info.generate_DF(data))
#Extraemos solo informacion importante
df = pd.concat(dataframes, sort=False)[['id','title','portrait_url','url']]

#df = 
#df.loc[df['id'] == 'eff479e3-dab9-38e3-b814-4e8b49f371d6']
jj = df.loc[df['portrait_url'] != '']
jj =jj.drop_duplicates(subset=['id'])

In [23]:
jj.to_csv('data/dic_products/products.csv', encoding='utf-8')